# 600_Cosine_Similarity

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import json
from openai import OpenAI


In [2]:
with open("C:/Users/danie/OneDrive/Desktop/openai_youtube_api_key.txt") as f:
    api_key = f.readline()

oai_client = OpenAI(api_key=api_key)

In [3]:
cleaned = pd.read_csv('../data/cleaned/channel_subset_with_comments.csv')
cleaned = cleaned[cleaned.channel_title.isin(["TYT's The Conversation", "Turning Point USA", 'TMM', 'Triggered Newscast'])].reset_index(drop=True)
cleaned.head()

,channel_id,channel_title,affiliation,video_id,video_title,video_description,comment_text,comment_id
0,UCLmEFIwfG3HSru_nUgOktGg,Triggered Newscast,R,__xDde9kpw0,Kevin McCarthy Nearly Defeats Nancy Pelosi in ...,#McCarthy #KevinMcCarthy #DrainTheSwamp\nDespi...,We the people need to start really fighting th...,UgzpCd27ArM_08w1XPh4AaABAg
1,UCLmEFIwfG3HSru_nUgOktGg,Triggered Newscast,R,__xDde9kpw0,Kevin McCarthy Nearly Defeats Nancy Pelosi in ...,#McCarthy #KevinMcCarthy #DrainTheSwamp\nDespi...,Bullshit. 2020 Election Fraud is off the hook....,Ugz4dCGe1U3YnWbXoDp4AaABAg
2,UCLmEFIwfG3HSru_nUgOktGg,Triggered Newscast,R,__xDde9kpw0,Kevin McCarthy Nearly Defeats Nancy Pelosi in ...,#McCarthy #KevinMcCarthy #DrainTheSwamp\nDespi...,How can the democrats commit political suicide...,Ugwlro6l4FK4FZ0PLoF4AaABAg
3,UCLmEFIwfG3HSru_nUgOktGg,Triggered Newscast,R,__xDde9kpw0,Kevin McCarthy Nearly Defeats Nancy Pelosi in ...,#McCarthy #KevinMcCarthy #DrainTheSwamp\nDespi...,I’m sure they made sure it was too close to co...,UgxARO3zib2Ve527Obd4AaABAg
4,UCLmEFIwfG3HSru_nUgOktGg,Triggered Newscast,R,__xDde9kpw0,Kevin McCarthy Nearly Defeats Nancy Pelosi in ...,#McCarthy #KevinMcCarthy #DrainTheSwamp\nDespi...,She’s a nasty mean drunk,UgzOZqdBWxXk-ixJQLV4AaABAg


In [4]:
cleaned.channel_title.unique()

array(['Triggered Newscast', 'Turning Point USA', 'TMM',
       "TYT's The Conversation"], dtype=object)

In [5]:
affils = ['progressive', 'conservative']
model_sizes = [5**n for n in range(2, 6)]

Get videos which have been used for training for each model

In [28]:
used_titles = {}
for affil in affils:
    for ms in model_sizes:
        used_titles[(affil, ms)] = []
        file_name = f"{affil}_size_{ms}_train.jsonl" 
        with open(f'../data/final_ft_datasets/{file_name}', 'r') as file:
            lines = file.readlines()
            for line in lines:
                start_ind = line.rindex('You just watched the youtube video ') + len('You just watched the youtube video ') + 1
                end_index = line.index(' with the description ') - 1
                used_titles[(affil, ms)].append(line[start_ind:end_index])
        used_titles[(affil, ms)] = list(set(used_titles[(affil, ms)]))
            

Get new videos for models to comment on.

In [29]:
import random
def sample_rows(group):
    return group.sample(min(len(group), 250)) 

videos_to_comment_on = {}
for affil in affils:
    for ms in model_sizes:
        affil_label = 'L' if affil == 'progressive' else 'R'
        unused_titles = cleaned[~cleaned['video_title'].isin(used_titles[(affil, ms)])]
        unused_titles = unused_titles[unused_titles.affiliation == affil_label]
        unused_titles = unused_titles[unused_titles.channel_title.isin(["TYT's The Conversation", "Turning Point USA"])]
        counts = unused_titles.video_title.value_counts()
        # Filter the DataFrame to keep only rows where the column value occurs at least 10 times
        filtered_df = unused_titles[unused_titles.video_title.isin(counts[counts >= 150].index)]
        # Apply the function to each group within the specified column
        filtered_df = filtered_df.groupby('video_title', group_keys=False).apply(sample_rows)
        
        unique_groups = filtered_df['video_title'].unique()

        # If there are fewer than 4 unique groups, keep all of them
        if len(unique_groups) <= 4:
            groups_to_keep = unique_groups
        else:
            # Randomly select 4 groups to keep
            np.random.shuffle(unique_groups)
            groups_to_keep = unique_groups[:4]

        # Filter the DataFrame to keep only rows from the selected groups
        filtered_df = filtered_df[filtered_df['video_title'].isin(groups_to_keep)]
        videos_to_comment_on[(affil, ms)] = filtered_df

In [113]:
for model in videos_to_comment_on:
    print(videos_to_comment_on[model].video_title.value_counts())

video_title
INSANE Interviews with MAGA Supporters                250
Should Medicare for All be Forced to a Floor Vote?    250
Trump’s FINAL Play to Steal the Election              250
Illegal Voter Purges in GEORGIA                       210
Name: count, dtype: int64
video_title
Trump’s FINAL Play to Steal the Election    250
Illegal Voter Purges in GEORGIA             210
Name: count, dtype: int64
video_title
Trump’s FINAL Play to Steal the Election    250
Name: count, dtype: int64
video_title
Trump’s FINAL Play to Steal the Election    250
Name: count, dtype: int64
video_title
Sex And Gender Are Directly Related                                                               211
Candace Owens CRUSHES Modern Feminism | Why Feminists Demonize Men Instead Of Empowering Women    203
We’ve Had Enough!! [Real America Episode 29]                                                      201
The Hodge Twins On Being Called "Sellouts" | Always Stand Up For Your Beliefs                     164
Name

We also need a dictionary for each model to comment on a video from the other channels

In [30]:
videos_to_comment_on_other_channels = {}
for affil in affils:
    for ms in model_sizes:
        videos_to_comment_on_other_channels[(affil, ms)] = {}
        affil_label = 'L' if affil == 'progressive' else 'R'
        possible_videos = cleaned[~cleaned.channel_title.isin(["TYT's The Conversation", "Turning Point USA"])]
        possible_videos = possible_videos[possible_videos.affiliation == affil_label]
        title = random.sample(possible_videos.video_title.tolist(), k=1)[0]
        videos_to_comment_on_other_channels[(affil, ms)]['video_title'] = title
        videos_to_comment_on_other_channels[(affil, ms)]['video_description'] = possible_videos[possible_videos.video_title == title].video_description.unique()[0]

videos_to_comment_on_other_channels

{('progressive', 25): {'video_title': 'Yet Another 10 Questions for Atheists',
  'video_description': 'The whole video:\nYes, I do have a Patreon account, thank you for asking:\nMy Twitter:\nMy facebook:\nHere’s my society6 store if you’re interested in my pretentious minimalist poster designs:\n'},
 ('progressive', 125): {'video_title': 'Was Jesus Really Buried?',
  'video_description': "The whole video:\nNonstampcollector's video:\nYes, I do have a Patreon account, thank you for asking:\nMy Twitter:\nMy facebook:\nHere’s my society6 store if you’re interested in my pretentious minimalist poster designs:\n"},
 ('progressive', 625): {'video_title': "Why the Universe Doesn't Need a Cause",
  'video_description': 'The whole video:\nYes, I do have a Patreon account, thank you for asking:\nMy Twitter:\nMy facebook:\nHere’s my society6 store if you’re interested in my pretentious minimalist poster designs:\n'},
 ('progressive',
  3125): {'video_title': 'Yet Another 10 Questions for Atheists

## Now let's get the responses

In [31]:
def format_question_for_chat_completion(title, description, affil):
    messages=[
        {"role": "system", "content": f"You are a {affil} American political commentator."},
        {"role": "user", "content": f"You just watched the youtube video '{title}' with the description '{description}'.\n\nGive your opinion on the subject matter."}
    ]
    return messages

In [32]:
affiliated_models_jobs = {
    'l' : {
        25: "ftjob-0y1Pfg6iOvUaWyZzYuRq4TJI",
        125: "ftjob-GBtoSSh1102sIQbZT954WNR0",
        625: "ftjob-HMEEAz2r4LVmtdFIo5zHRRWC",
        3125: "ftjob-Pop4MECv0nHNdBcYZYy0rSjb",
        15625: "ftjob-AEqdzMVWiZhMxv8Y6m3b1pHE"
    },
    
    'r' : {
        25: "ftjob-q5UHn926xz9g8wfdCPjFuK7U",
        125: "ftjob-dUQD3xBBWUQ33mA1AcTRUxSr",
        625: "ftjob-p9GPwKfqlV0Zs32kjwkay703",
        3125: "ftjob-gSeeH5p5VPXMA2AwyKqGs11S",
    }
}

In [33]:
affiliated_models = {
    affil : {   k: oai_client.fine_tuning.jobs.retrieve(v).fine_tuned_model for (k,v) in model_dic.items()  }
        for affil, model_dic in affiliated_models_jobs.items()
}

affiliated_models

{'l': {25: 'ft:gpt-3.5-turbo-0613:personal::8HuIhUqH',
  125: 'ft:gpt-3.5-turbo-0613:personal::8Hu2dTTZ',
  625: 'ft:gpt-3.5-turbo-0613:personal::8HteyfJc',
  3125: 'ft:gpt-3.5-turbo-0613:personal::8HsYApwC',
  15625: 'ft:gpt-3.5-turbo-0613:personal::8HizTQw1'},
 'r': {25: 'ft:gpt-3.5-turbo-0613:personal::8HzsJtSa',
  125: 'ft:gpt-3.5-turbo-0613:personal::8Hzc3hWd',
  625: 'ft:gpt-3.5-turbo-0613:personal::8HxCZnqf',
  3125: 'ft:gpt-3.5-turbo-0613:personal::8HviUlwy'}}

In [35]:
videos_comments_and_responses = {}
for affil in affils:
    for ms in model_sizes:
        affil_label = 'l' if affil == 'progressive' else 'r'
        videos_comments_and_responses[(affil, ms)] = []
        rows = videos_to_comment_on[(affil, ms)]
        for video in rows.video_title.unique():
            video_dic = {}
            video_dic['video_title'] = video
            video_rows = rows[rows.video_title == video]
            video_dic['human_comments'] = video_rows.comment_text.tolist()
            message_list = format_question_for_chat_completion(video_rows.video_title, video_rows.video_description, affil)
            n = video_rows.shape[0] 
            if n > 128:
                second_run = n-128
                response1 = oai_client.chat.completions.create(
                    model=affiliated_models[affil_label][ms],
                    messages=message_list,
                    n=128
                )
                temp1 = [choice.message.content for choice in response1.choices]
                response2 = oai_client.chat.completions.create(
                    model=affiliated_models[affil_label][ms],
                    messages=message_list,
                    n=second_run
                )
                temp2 = [choice.message.content for choice in response2.choices]
                video_dic['llm_comments'] = temp1+temp2
            else:
                response = oai_client.chat.completions.create(
                    model=affiliated_models[affil_label][ms],
                    messages=message_list,
                    n=n
                )
                video_dic['llm_comments'] = [choice.message.content for choice in response.choices]
            videos_comments_and_responses[(affil, ms)].append(video_dic)

            

In [89]:
for affil in affils:
    for ms in model_sizes:
        affil_label = 'l' if affil == 'progressive' else 'r'
        info = videos_to_comment_on_other_channels[(affil, ms)]
        message_list = format_question_for_chat_completion(info['video_title'], video_rows['video_description'], affil)
        response1 = oai_client.chat.completions.create(
            model=affiliated_models[affil_label][ms],
            messages=message_list,
            n=125
        )
        temp1 = [choice.message.content for choice in response1.choices]
        response2 = oai_client.chat.completions.create(
            model=affiliated_models[affil_label][ms],
            messages=message_list,
            n=125
        )
        temp2 = [choice.message.content for choice in response2.choices]
        videos_to_comment_on_other_channels[(affil, ms)]['llm_comments'] = temp1+temp2

Let's save these to a file.

In [36]:
import json
output_dic = {}
for affil in affils:
    output_dic[affil] = {}
    for ms in model_sizes:
        output_dic[affil][ms] = videos_comments_and_responses[(affil, ms)]
with open("../data/experiment_rerun/consine_similarity_responses.json", 'w') as file:
    json.dump(output_dic, file)

In [93]:
videos_comments_and_responses_other_channels = videos_to_comment_on_other_channels

output_dic = {}
for affil in affils:
    output_dic[affil] = {}
    for ms in model_sizes:
        output_dic[affil][ms] = videos_comments_and_responses_other_channels[(affil, ms)]
        

with open("../data/experiment_rerun/consine_similarity_responses_other_channels.json", 'w') as file:
    json.dump(output_dic, file)

# Get the embeddings

In [39]:
from copy import deepcopy

videos_comments_and_responses_embedded = deepcopy(videos_comments_and_responses)

In [37]:
def get_embedding(text, model="text-embedding-3-small"): # model = "deployment_name"
    return oai_client.embeddings.create(input = [text], model=model).data[0].embedding

In [21]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [41]:
from openai import BadRequestError
for affil in affils:
    for ms in model_sizes:
        skipped_human = 0
        skipped_llm = 0
        for i in range(len(videos_comments_and_responses_embedded[(affil, ms)])):
            human_arr = []
            for x in videos_comments_and_responses_embedded[(affil, ms)][i]['human_comments']:
                try:
                    human_arr.append(get_embedding(x))
                except BadRequestError as e:
                    skipped_human += 1
                    continue
            videos_comments_and_responses_embedded[(affil, ms)][i]['human_comments'] = human_arr
            
            llm_arr = []
            for x in videos_comments_and_responses_embedded[(affil, ms)][i]['llm_comments']:
                try:
                    llm_arr.append(get_embedding(x))
                except BadRequestError as e:
                    skipped_llm += 1      
                    continue
            videos_comments_and_responses_embedded[(affil, ms)][i]['llm_comments'] = llm_arr
        print(f"For Model: {affil},{ms}")
        print(f"{skipped_human} comments skipped, {skipped_llm} comments skipped")
        

For Model: progressive,25
0 comments skipped, 0 comments skipped
For Model: progressive,125
0 comments skipped, 0 comments skipped
For Model: progressive,625
0 comments skipped, 0 comments skipped
For Model: progressive,3125
0 comments skipped, 0 comments skipped
For Model: conservative,25
0 comments skipped, 1 comments skipped
For Model: conservative,125
0 comments skipped, 0 comments skipped
For Model: conservative,625
0 comments skipped, 0 comments skipped
For Model: conservative,3125
0 comments skipped, 0 comments skipped


In [42]:
output_dic = {}
for affil in affils:
    output_dic[affil] = {}
    for ms in model_sizes:
        output_dic[affil][ms] = videos_comments_and_responses_embedded[(affil, ms)]
with open("../data/experiment_rerun/consine_similarity_embedded.json", 'w') as file:
    json.dump(output_dic, file)

# It broke. Lets load the data again

In [43]:
with open("../data/experiment_rerun/consine_similarity_embedded.json", 'r') as file:
    videos_comments_and_responses_embedded= json.load(file)

In [44]:
with open("../data/experiment_rerun/consine_similarity_responses_other_channels.json", 'r') as file:
    videos_comments_and_responses_other_channels_embedded = json.load(file)

In [21]:
from openai import BadRequestError
for affil in affils:
    for ms in model_sizes:
        skipped_llm = 0
        llm_arr = []
        for x in videos_comments_and_responses_other_channels_embedded[affil][str(ms)]['llm_comments']:
            try:
                llm_arr.append(get_embedding(x))
            except BadRequestError as e:
                skipped_llm += 1      
                continue
        videos_comments_and_responses_other_channels_embedded[affil][str(ms)]['llm_comments'] = llm_arr
        print(f"For Model: {affil},{ms}")
        print(f"{skipped_llm} comments skipped")
        

For Model: progressive,25
1 comments skipped
For Model: progressive,125
0 comments skipped
For Model: progressive,625
0 comments skipped
For Model: progressive,3125
0 comments skipped
For Model: conservative,25
0 comments skipped
For Model: conservative,125
0 comments skipped
For Model: conservative,625
0 comments skipped
For Model: conservative,3125
0 comments skipped


In [169]:
with open("../data/experiment_rerun/consine_similarity_other_channels_embedded.json", 'w') as file:
    json.dump(videos_comments_and_responses_other_channels_embedded, file)

In [45]:
with open("../data/experiment_rerun/consine_similarity_other_channels_embedded.json", 'r') as file:
    videos_comments_and_responses_other_channels_embedded = json.load(file)

In [104]:
cs_part1_df = pd.DataFrame(columns = ['affiliation',
                                      'size',
                                      'trial_id',
                                      'similarity',
                                      'similarity_type'])
cs_part1_df

,affiliation,size,trial_id,similarity,similarity_type


In [105]:
from random import sample
for affil in affils:
    for ms in model_sizes:
        data = {
                'trial_id' : [],
                'similarity' : [],
                'similarity_type' : []   
            }
        for trial_id in range(1, 1001):
            
            selected_video = sample(range(len(videos_comments_and_responses_embedded[affil][str(ms)])), 1)[0]
            human_comment_for_comparison = sample(videos_comments_and_responses_embedded[affil][str(ms)][selected_video]['human_comments'], k=1)
            human_comment_list = sample([x for x in videos_comments_and_responses_embedded[affil][str(ms)][selected_video]['human_comments'] if x != human_comment_for_comparison], 100)
            llm_comment_list = sample(videos_comments_and_responses_embedded[affil][str(ms)][selected_video]['llm_comments'], 100)
            
            data['trial_id'].append(trial_id)
            data['similarity'].append(np.mean([cosine_similarity(human_comment_for_comparison, x) for x in human_comment_list]))
            data['similarity_type'].append('human')
            data['trial_id'].append(trial_id)
            data['similarity'].append(np.mean([cosine_similarity(human_comment_for_comparison, x) for x in llm_comment_list]))
            data['similarity_type'].append('synthetic')
            
        df = pd.DataFrame(data)
        df['affiliation'] = affil
        df['size'] = ms
        cs_part1_df = pd.concat([cs_part1_df, df])


C:\Users\danie\AppData\Local\Temp\ipykernel_12228\3215319555.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cs_part1_df = pd.concat([cs_part1_df, df])


In [108]:
cs_part1_df

,affiliation,size,trial_id,similarity,similarity_type
0,progressive,25,1,0.182299,human
1,progressive,25,1,0.174904,synthetic
2,progressive,25,2,0.173855,human
3,progressive,25,2,0.182361,synthetic
4,progressive,25,3,0.143762,human
...,...,...,...,...,...
1995,conservative,3125,998,0.109620,synthetic
1996,conservative,3125,999,0.275310,human
1997,conservative,3125,999,0.190809,synthetic
1998,conservative,3125,1000,0.179381,human


In [190]:
cs_part1_df.to_csv("../data/experiment_rerun/cosine_similarity_human_synthetic_comparisons.csv")

In [170]:
import statsmodels.formula.api as smf

# NB. unlike sm.OLS, there is "intercept" term is included here
lm = smf.ols(formula="similarity ~ C(similarity_type)", data=cs_part1_df).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:             similarity   R-squared:                       0.061
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     1040.
Date:                Wed, 06 Mar 2024   Prob (F-statistic):          4.04e-221
Time:                        09:39:39   Log-Likelihood:                 27487.
No. Observations:               16000   AIC:                        -5.497e+04
Df Residuals:                   15998   BIC:                        -5.496e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

In [161]:
cs_part2_df = pd.DataFrame(columns = ['affiliation',
                                      'size',
                                      'trial_id',
                                      'similarity',
                                      'similarity_type'
                                      ])
cs_part2_df

,affiliation,size,trial_id,similarity,similarity_type


In [126]:
import random
def sample_rows(group):
    return group.sample(min(len(group), 250)) 

videos_to_comment_on_2 = {}
for affil in affils:
    for ms in model_sizes:
        affil_label = 'L' if affil == 'progressive' else 'R'
        unused_titles = cleaned[~cleaned['video_title'].isin(used_titles[(affil, ms)])]
        unused_titles = unused_titles[unused_titles.affiliation == affil_label]
        filtered_df = unused_titles[unused_titles.channel_title.isin(["TYT's The Conversation", "Turning Point USA"])]
        '''
        counts = unused_titles.video_title.value_counts()
        # Filter the DataFrame to keep only rows where the column value occurs at least 10 times
        filtered_df = unused_titles[unused_titles.video_title.isin(counts[counts >= 5].index)]
        # Apply the function to each group within the specified column
        filtered_df = filtered_df.groupby('video_title', group_keys=False).apply(sample_rows)
        
        unique_groups = filtered_df['video_title'].unique()

        # If there are fewer than 4 unique groups, keep all of them
        if len(unique_groups) <= 4:
            groups_to_keep = unique_groups
        else:
            # Randomly select 4 groups to keep
            np.random.shuffle(unique_groups)
            groups_to_keep = unique_groups[:4]

        # Filter the DataFrame to keep only rows from the selected groups
        filtered_df = filtered_df[filtered_df['video_title'].isin(groups_to_keep)]'''
        videos_to_comment_on_2[(affil, ms)] = filtered_df

In [127]:
for model in videos_to_comment_on:
    print(videos_to_comment_on_2[model].video_title.value_counts())

video_title
Should Medicare for All be Forced to a Floor Vote?    506
INSANE Interviews with MAGA Supporters                475
Trump’s FINAL Play to Steal the Election              288
Illegal Voter Purges in GEORGIA                       210
Who Is Behind QANON?                                  128
                                                     ... 
Drag Queen Wants to SAVE the Environment                6
Banks Continue To FAIL Minority Communities             5
The Threat of Right-wing Terrorists                     4
Georgia is the Newest BATTLEGROUND State                3
The Long Battle For Police Accountability               2
Name: count, Length: 62, dtype: int64
video_title
Trump’s FINAL Play to Steal the Election                        288
Illegal Voter Purges in GEORGIA                                 210
Who Is Behind QANON?                                            128
Racial JUSTICE Under the Biden Administration                   118
First Progressive News Show 

In [149]:
for affil in affils:
    for ms in model_sizes:
        if len(videos_comments_and_responses_embedded[affil][str(ms)]) < 2:
            affil_label = 'l' if affil == 'progressive' else 'r'
            other_video = sample([x for x in list(videos_to_comment_on_2[(affil, ms)].video_title.unique()) if x != videos_comments_and_responses_embedded[affil][str(ms)][0]['video_title']], 1)[0]
            other_video_description = list(cleaned.loc[cleaned.video_title == other_video, :].video_description.unique())[0]
            message_list = format_question_for_chat_completion(other_video, other_video_description, affil)
            response1 = oai_client.chat.completions.create(
                model=affiliated_models[affil_label][ms],
                messages=message_list,
                n=125
            )
            temp1 = [choice.message.content for choice in response1.choices]
            response2 = oai_client.chat.completions.create(
                model=affiliated_models[affil_label][ms],
                messages=message_list,
                n=125
            )
            temp2 = [choice.message.content for choice in response2.choices]
            videos_comments_and_responses_embedded[affil][str(ms)].append({
                'video_title' : other_video,
                'llm_comments' : [get_embedding(x) for x in temp1+temp2]
            })

In [147]:
videos_comments_and_responses_embedded['progressive']['25'][0].keys()

dict_keys(['video_title', 'human_comments', 'llm_comments'])

In [217]:
def get_synthetic_group(group):
    return group[group['similarity_type'] == 'synthetic']

cs_part2_df = pd.DataFrame(columns = ['affiliation',
                                      'size',
                                      'trial_id',
                                      'similarity',
                                      'similarity_type'
                                      ])
for affil in affils:
    for ms in model_sizes:
        data = {
                'trial_id' : [],
                'similarity' : [],
                'similarity_type' : []   
            }
        
        #Part 1 - get comparison for same video.
        df_1 = cs_part1_df.groupby(['affiliation', 'size', 'similarity_type']).apply(get_synthetic_group).reset_index(drop=True)
        df_1 = df_1[df_1['size'] == ms]
        df_1 = df_1[df_1.affiliation == affil]
        df_1['similarity_type'] = 'same_video'
        cs_part2_df = pd.concat([cs_part2_df, df_1])
        
        for trial_id in range(1, 1001):
            #Part 2 - get comparison for different video from same channel.
            data = {
                'trial_id' : [],
                'similarity' : [],
                'similarity_type' : []   
            }
            selected_video = sample(range(len(videos_comments_and_responses_embedded[affil][str(ms)])), 1)[0]
            while not 'human_comments' in videos_comments_and_responses_embedded[affil][str(ms)][selected_video].keys():
                selected_video = sample(range(len(videos_comments_and_responses_embedded[affil][str(ms)])), 1)[0]
            comparison_video = sample(range(len(videos_comments_and_responses_embedded[affil][str(ms)])), 1)[0]
            while comparison_video == selected_video:
                comparison_video = sample(range(len(videos_comments_and_responses_embedded[affil][str(ms)])), 1)[0]
            human_comment_for_comparison = sample(videos_comments_and_responses_embedded[affil][str(ms)][selected_video]['human_comments'], k=1)
            llm_comment_list = sample(videos_comments_and_responses_embedded[affil][str(ms)][comparison_video]['llm_comments'], 100)
            data['trial_id'].append(trial_id)
            data['similarity'].append(np.mean([cosine_similarity(human_comment_for_comparison, x) for x in llm_comment_list]))
            data['similarity_type'].append('same_channel')    
            df_2 = pd.DataFrame(data)
            df_2['affiliation'] = affil
            df_2['size'] = ms
            
            
            #Part 3 - get comparison for different channel
            data = {
                'trial_id' : [],
                'similarity' : [],
                'similarity_type' : []   
            }
            selected_video = sample(range(len(videos_comments_and_responses_embedded[affil][str(ms)])), 1)[0]
            while not 'human_comments' in videos_comments_and_responses_embedded[affil][str(ms)][selected_video].keys():
                selected_video = sample(range(len(videos_comments_and_responses_embedded[affil][str(ms)])), 1)[0]
            human_comment_for_comparison = sample(videos_comments_and_responses_embedded[affil][str(ms)][selected_video]['human_comments'], k=1) 
            comparison_video = sample(range(len(videos_comments_and_responses_other_channels_embedded[affil][str(ms)])), 1)[0]  
            llm_comment_list = sample(videos_comments_and_responses_other_channels_embedded[affil][str(ms)]['llm_comments'], 100)
            data['trial_id'].append(trial_id)
            data['similarity'].append(np.mean([cosine_similarity(human_comment_for_comparison, x) for x in llm_comment_list]))
            data['similarity_type'].append('different_channel')    
            df_3 = pd.DataFrame(data)
            df_3['affiliation'] = affil
            df_3['size'] = ms
            
            cs_part2_df=pd.concat([cs_part2_df, df_2, df_3])

C:\Users\danie\AppData\Local\Temp\ipykernel_12228\3143676611.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cs_part2_df = pd.concat([cs_part2_df, df_1])


In [218]:
cs_part2_df.to_csv("../data/experiment_rerun/cosine_similarity_synthetic_type_comparisons.csv")

In [209]:
temp = cs_part2_df.groupby(['affiliation', 'size', 'similarity_type']).agg({'similarity':'mean'})

In [210]:
# Assuming your DataFrame is named df
pivoted_df = temp.pivot_table(
    index=['size', 'similarity_type'],  # size and similarity_type as the new index
    columns='affiliation',              # affiliation as the new columns
    values='similarity'                 # similarity as the values to fill the DataFrame
).swaplevel(0, 1, axis=0)               # swap levels to make similarity_type the outer index

custom_order = ['same_video', 'same_channel', 'different_channel']

# Reindex the DataFrame according to the custom order
# You would do this on the level of the index where 'similarity_type' is
pivoted_df = pivoted_df.reindex(custom_order, level='similarity_type')

# Sort the index to have a nice hierarchical order
#pivoted_df = pivoted_df.sort_index(level=0)


In [211]:
pivoted_df

affiliation             conservative  progressive
similarity_type   size                           
same_video        25        0.183933     0.202204
                  125       0.184084     0.198694
                  625       0.175512     0.197850
                  3125      0.173395     0.195701
same_channel      25        0.157763     0.180349
                  125       0.161564     0.191112
                  625       0.165217     0.135790
                  3125      0.161635     0.139951
different_channel 25        0.166106     0.163773
                  125       0.174330     0.144167
                  625       0.161526     0.167137
                  3125      0.170484     0.137028